In [ ]:
import pandas as pd
import numpy as np
from pyproj import Transformer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor, Pool
import optuna

In [ ]:
# Пример данных о локациях
data = {
    'latitude': [40.7128, 40.7306, 40.7484, 40.7580, 40.7688, 40.7788, 40.7888, 40.7988],
    'longitude': [-74.0060, -73.9352, -73.9857, -73.9857, -73.9681, -73.9581, -73.9481, -73.9381],
    'idealness': [0.8, 0.9, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]  # Примерные оценки идеальности
}

df = pd.DataFrame(data)

# Создаем трансформер для проекции Меркатора
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)

# Преобразуем географические координаты в плоские
df['x'], df['y'] = transformer.transform(df['longitude'].values, df['latitude'].values)

# Функция для расчета манхэттенского расстояния
def manhattan_distance(x1, y1, x2, y2):
    return abs(x2 - x1) + abs(y2 - y1)

# Расчет матрицы манхэттенских расстояний
def compute_manhattan_distances(df):
    n = len(df)
    distances = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            distances[i, j] = manhattan_distance(df.loc[i, 'x'], df.loc[i, 'y'], df.loc[j, 'x'], df.loc[j, 'y'])
    return distances

# Вычисляем матрицу манхэттенских расстояний
distances_matrix = compute_manhattan_distances(df)

# Добавляем расстояния до всех локаций в качестве признаков
distances_df = pd.DataFrame(distances_matrix, columns=[f'distance_to_{j}' for j in range(len(df))])
df = pd.concat([df, distances_df], axis=1)

# Разделяем данные на обучающую и тестовую выборки
X = df.drop(columns=['latitude', 'longitude', 'idealness'])
y = df['idealness']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создаем объект Pool для CatBoost
train_pool = Pool(X_train, y_train)
test_pool = Pool(X_test, y_test)

# Функция для оптимизации гиперпараметров с использованием Optuna
def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 50, 200),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-5, 10),
        'random_strength': trial.suggest_loguniform('random_strength', 1e-5, 10),
        'bagging_temperature': trial.suggest_loguniform('bagging_temperature', 0.01, 10),
        'od_type': 'Iter',
        'od_wait': 10,
        'verbose': 0,
        'random_state': 42
    }

    model = CatBoostRegressor(**params)
    model.fit(train_pool, eval_set=test_pool, early_stopping_rounds=10)
    y_pred = model.predict(test_pool)
    mse = mean_squared_error(y_test, y_pred)
    return mse

# Оптимизация гиперпараметров с использованием Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Лучшие параметры
best_params = study.best_params
print(f"Лучшие параметры: {best_params}")

# Обучение модели с лучшими параметрами
best_model = CatBoostRegressor(**best_params, verbose=0)
best_model.fit(train_pool, eval_set=test_pool, early_stopping_rounds=10)

# Делаем предсказания на тестовой выборке
y_pred_best = best_model.predict(test_pool)

# Оцениваем качество модели
mse_best = mean_squared_error(y_test, y_pred_best)
print(f"Среднеквадратичная ошибка для CatBoost с оптимизированными параметрами: {mse_best}")

# Кросс-валидация
cv_scores = cross_val_score(best_model, X, y, cv=5, scoring='neg_mean_squared_error')
mse_cv = -cv_scores.mean()
print(f"Среднеквадратичная ошибка после кросс-валидации: {mse_cv}")

# Предсказываем идеальность для новых локаций
new_locations = pd.DataFrame({
    'latitude': [40.7530, 40.7688],
    'longitude': [-73.9824, -73.9681]
})

new_locations['x'], new_locations['y'] = transformer.transform(new_locations['longitude'].values, new_locations['latitude'].values)

# Вычисляем матрицу манхэттенских расстояний для новых локаций
new_distances_matrix = np.zeros((len(new_locations), len(df)))
for i in range(len(new_locations)):
    for j in range(len(df)):
        new_distances_matrix[i, j] = manhattan_distance(new_locations.loc[i, 'x'], new_locations.loc[i, 'y'], df.loc[j, 'x'], df.loc[j, 'y'])

# Добавляем расстояния до всех локаций в качестве признаков
new_distances_df = pd.DataFrame(new_distances_matrix, columns=[f'distance_to_{j}' for j in range(len(df))])
new_locations = pd.concat([new_locations, new_distances_df], axis=1)

# Предсказания с использованием CatBoost с оптимизированными параметрами
new_locations_pred_best = best_model.predict(new_locations.drop(columns=['latitude', 'longitude']))
print(f"Предсказанная идеальность для новых локаций с использованием CatBoost с оптимизированными параметрами: {new_locations_pred_best}")